# Run this beast!

In [ ]:

import os
import shutil
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pyemu
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import flopy

import sys
sys.path.append("..")
# import pre-prepared convenience functions
import herebedragons as hbd

In [ ]:
# specify the temporary working folder
t_d = os.path.join('freyberg6_da_template')

if not os.path.exists(t_d):
    raise Exception("you need to run the 'freyberg_da_prep.ipynb' notebook")



In [ ]:
pst = pyemu.Pst(os.path.join(t_d,"freyberg_mf6.pst"))

In [ ]:
pst.pestpp_options['ies_parameter_ensemble'] = 'prior_pe.jcb'
pst.pestpp_options["ies_num_reals"] = 20
num_workers = 10
pst.control_data.noptmax = 2
pst.write(os.path.join(t_d, 'freyberg_mf6.pst'))
m_d = os.path.join('master_da')

In [ ]:
pyemu.os_utils.start_workers(t_d, # the folder which contains the "template" PEST dataset
                            'pestpp-da', #the PEST software version we want to run
                            'freyberg_mf6.pst', # the control file to use with PEST
                            num_workers=num_workers, #how many agents to deploy
                            worker_root='.', #where to deploy the agent directories; relative to where python is running
                            master_dir=m_d, #the manager directory
                            )

Like you probably have realized, all things get more complicated with sequential estimation, this includes post processing as well...

Let's load the global obs ensembles

In [ ]:
files = [f for f in os.listdir(m_d) if "global" in f and "oe" in f]
oes = {int(f.split(".")[2]):pd.read_csv(os.path.join(m_d,f),index_col=0) for f in files}

In [ ]:
files = [f for f in os.listdir(m_d) if ".0.obs.csv" in f]
pr_oes = {int(f.split(".")[1]):pd.read_csv(os.path.join(m_d,f),index_col=0) for f in files}

In [ ]:
files = [f for f in os.listdir(m_d) if ".{0}.obs.csv".format(pst.control_data.noptmax) in f]
pt_oes = {int(f.split(".")[1]):pd.read_csv(os.path.join(m_d,f),index_col=0) for f in files}

In [ ]:
otab = pd.read_csv(os.path.join(m_d,"obs_cycle_table.csv"),index_col=0)
wtab = pd.read_csv(os.path.join(m_d,"weight_cycle_table.csv"),index_col=0)

In [ ]:
obs = pst.observation_data
obs = obs.loc[pst.nnz_obs_names,:]
obs

In [ ]:
for o in pst.nnz_obs_names:
    fig,ax = plt.subplots(1,1)
    for kper,oe in pr_oes.items():
        ax.scatter([kper]*oe.shape[0],oe.loc[:,o].values,marker=".",c="0.5",alpha=0.5)
    for kper,oe in pt_oes.items():
        ax.scatter([kper]*oe.shape[0],oe.loc[:,o].values,marker=".",c="b",alpha=0.5)
        
    
    ovals = otab.loc[o,:].values
    wvals = wtab.loc[o,:].values
    ovals[wvals==0] = np.nan
    ylim = ax.get_ylim()
    ax.scatter(otab.columns.values,ovals,marker='.',c='r')
    ax.set_ylim(ylim)
    ax.set_title(o,loc="left")
    ax.set_xlabel("kper")
    
    plt.tight_layout()
